# Introduction to Tensorflow

- __Git Installation:__ [git-scm](https://git-scm.com/book/en/v2/Getting-Started-Installing-Git)

- __Required Packages:__
  - tensorflow==2.7.0 NOT but 2.6.0
  - scikit-learn==1.0.1
  - pandas==1.1.5
  - matplotlib==3.2.2
  - seaborn==0.11.2

## Imports

Start with importing the required packages with selected versions. [TensorFlow](https://www.tensorflow.org/) is imported with `tf` for convention and ease of use. [`numpy`](https://numpy.org) is imported to represent data as arrays and to optimize numerical operations. The framework [`keras`](https://keras.io/) is also imported to use in order to build neural networks as a sequence of layers.

In [9]:
# To check the tensorflow package installed on the environment
!pip show tensorflow

Name: tensorflow
Version: 2.6.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: c:\programdata\anaconda3\envs\tf-gpu-dlai\lib\site-packages
Requires: numpy, typing-extensions, wrapt, tensorflow-estimator, six, gast, flatbuffers, tensorboard, h5py, astunparse, opt-einsum, termcolor, grpcio, clang, wheel, keras-preprocessing, keras, google-pasta, absl-py, protobuf
Required-by: 


In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow import keras

print(tf.__version__)

In [1]:
import tensorflow as tf
print("TensorFlow Version:", tf.__version__)

# uncomment to see where the variables get placed
# tf.debugging.set_log_device_placement(True)

#print("Available GPU:",tf.test.gpu_device_name())
gpus = tf.config.list_physical_devices('GPU')
for gpu in gpus:
    print("Name:", gpu.name, "  Type:", gpu.device_type)

TensorFlow Version: 2.6.0
Name: /physical_device:GPU:0   Type: GPU


### 2) Tensors

#### 2.1. Variables
A _TensorFlow variable_ is the recommended way to represent shared, persistent state which the program manipulates. Variables are created and tracked via the `tf.Variable` class. A `tf.Variable` represents a tensor whose value can be changed by running operations on it and will have the same `dtype` as the initialization value. Specific operations allow one to read and modify the values of this tensor. Higher level libraries like `tf.keras` use `tf.Variable` to store model parameters. ([Variables](https://www.tensorflow.org/guide/variable)).

In [2]:
# scalar (rank 0)
string = tf.Variable("a", tf.string)
int_var = tf.Variable(128, tf.int16)
float_var = tf.Variable(3.147, tf.float64)
bool_var = tf.Variable(False)
complex_var = tf.Variable(5 + 4j)

# higher rank
rank_1 = tf.Variable(["a","b","c","d"], tf.string)
rank_2 = tf.Variable([["a","b"],["c","d"],["e","f"]], tf.string)

#### 2.2. Rank, Dtype and Shape of Tensor

A variable looks and acts like a tensor, and, in fact, is a data structure backed by a `tf.Tensor`. Like tensors, they have a `dtype` and a `shape`, and can be exported to NumPy. Also eash tensor has a `rank` which is the number of dimensions of the tensor.

In [3]:
# rank/degree of a tensor
print("Ranks:")
print("- string:",tf.rank(string))
print("- rank_1:",tf.rank(rank_1))
print("- rank_2:",tf.rank(rank_2))

# shape and dtype of a tensor
print("\nShape: ", rank_2.shape)
print("DType: ", rank_2.dtype)
print("As NumPy: ", rank_2.numpy())

Ranks:
- string: tf.Tensor(0, shape=(), dtype=int32)
- rank_1: tf.Tensor(1, shape=(), dtype=int32)
- rank_2: tf.Tensor(2, shape=(), dtype=int32)

Shape:  (3, 2)
DType:  <dtype: 'string'>
As NumPy:  [[b'a' b'b']
 [b'c' b'd']
 [b'e' b'f']]


#### 2.3. Reshape Tensor

Most tensor operations work on variables as expected, although variables cannot be reshaped. However, `tf.reshape` creates a new tensor with the desired shape if the total number of elements in the input tensor is preserved.

In [4]:
tensor1 = tf.ones([1,2,3]) # tf.ones() creates a tensor with the given shape full of ones 
tensor2 = tf.reshape(tensor1, [2,1,3]) # reshape tensor1 to shape [2,3,1]
tensor3 = tf.reshape(tensor1, [2,-1]) # -1 tells tensor to calculate the size of the dimension in that space
tensor4 = tf.reshape(tensor1, [-1]) # -1 tells tensor to calculate the size of the dimension in that space

print(tensor1)
print(tensor2)
print(tensor3)
print(tensor4)

tf.Tensor(
[[[1. 1. 1.]
  [1. 1. 1.]]], shape=(1, 2, 3), dtype=float32)
tf.Tensor(
[[[1. 1. 1.]]

 [[1. 1. 1.]]], shape=(2, 1, 3), dtype=float32)
tf.Tensor(
[[1. 1. 1.]
 [1. 1. 1.]], shape=(2, 3), dtype=float32)
tf.Tensor([1. 1. 1. 1. 1. 1.], shape=(6,), dtype=float32)


#### 2.4. Tensor Types

- Constant: `tf.constant` creates a constant tensor from a tensor-like object
- Variable: `tf.Variable` creates a variable with the same dtype as the initialization value.
- Placeholder
- SparseTensor

With the exception of `Variable`, all tensors are immutable and their value cannot change during the execution.

In [8]:
import numpy as np
# constant tensor with given dtype and shape
const_tensor = tf.constant([1, 2, 3, 4, 5, 6], dtype=tf.float64, shape=[2, 3])
num_array = np.array([[1, 2, 3], [4, 5, 6]])
const_tensor2 = tf.constant(num_array)
# variable tensor
var_tensor = tf.Variable(const_tensor)

print(const_tensor,const_tensor2,var_tensor)

tf.Tensor(
[[1. 2. 3.]
 [4. 5. 6.]], shape=(2, 3), dtype=float64) tf.Tensor(
[[1 2 3]
 [4 5 6]], shape=(2, 3), dtype=int32) <tf.Variable 'Variable:0' shape=(2, 3) dtype=float64, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]])>


#### 2.5. Assign and Duplicate Tensors
A tensor can be reassigned using `tf.Variable.assign`.  Calling `assign` does not (usually) allocate a new tensor; instead, the existing tensor's memory is reused. Assing operation does not allow to resize the tensor. However, creating new variables from existing variables duplicates the backing tensors. Two variables will not share the same memory.

In [5]:
a = tf.Variable([2.0, 3.0])
# create b based on the value of a
b = tf.Variable(a)
a.assign([5, 6])

# a and b are different
print(a.numpy()) # [5. 6.]
print(b.numpy()) # [2. 3.]

# there are other versions of assign
print(a.assign_add([2,3]).numpy())  # [7. 9.]
print(a.assign_sub([7,9]).numpy())  # [0. 0.]

[5. 6.]
[2. 3.]
[7. 9.]
[0. 0.]


#### 2.6. Evaluating Tensors
Since the tensors represent a partially completed computation, a session is run to evaluate the tensor. A simple way to do it is:

In [ ]:
with tf.Session() as sess: # creates a session using the default graph
    tensor_name.eval() # evaluates the tensor with the name `tensor_name`

#### 2.7. Tensors Lifecycles, Naming and Watching
In Python-based TensorFlow, `tf.Variable` instance have the same lifecycle as other Python objects. When there are no references to a variable it is automatically deallocated. Variables can also be named to track and debug. Two variables can be given the same name. Variable names are preserved when saving and loading models. By default, variables in models will acquire unique variable names automatically.

In [10]:
# different tensors have the same name
my_tensor1 = tf.Variable(const_tensor, name="Sample")
# new variable with the same name, but different value
my_tensor2 = tf.Variable(const_tensor + 1, name="Sample")

# these are elementwise-unequal, despite having the same name
print(a == b)

tf.Tensor([False False], shape=(2,), dtype=bool)


Although variables are important for differentiation, some variables will not need to be differentiated.  You can turn off gradients for a variable by setting `trainable` to false at creation. An example of a variable that would not need gradients is a training step counter.

In [6]:
step_counter = tf.Variable(1, trainable=False)

#### 2.8. Placing Variables and Tensors

For better performance, TensorFlow will attempt to place tensors and variables on the fastest device compatible with its `dtype`. This means most variables are placed on a GPU if one is available. However, one can override this.  In this snippet, place a float tensor and a variable on the CPU, even if a GPU is available (see the placement with `tf.debugging.set_log_device_placement(True)`). 

__Note:__ Although manual placement works, using [distribution strategies](distributed_training.ipynb) can be a more convenient and scalable way to optimize the computation.

In [2]:
with tf.device('CPU:0'):
    # create some tensors and place on CPU
    a = tf.Variable([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
    b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
    c = tf.matmul(a, b)
print(c)

Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op MatMul in device /job:localhost/replica:0/task:0/device:CPU:0
tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32)


In [3]:
with tf.device('CPU:0'):
    a = tf.Variable([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
    b = tf.Variable([[1.0, 2.0, 3.0]])

with tf.device('GPU:0'):
    # element-wise multiply
    k = a * b
print(k)

Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op DestroyResourceOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Mul in device /job:localhost/replica:0/task:0/device:GPU:0
tf.Tensor(
[[ 1.  4.  9.]
 [ 4. 10. 18.]], shape=(2, 3), dtype=float32)
